Creates the example for poc.

In [15]:
%run sql_magics.ipynb

In [16]:
%run plpgsql_helpers.ipynb

CREATE FUNCTION


In [17]:
%%sql

DROP TABLE IF EXISTS patch1 CASCADE;
DROP TABLE IF EXISTS patch2 CASCADE;

NOTICE:  drop cascades to 2 other objects
DETAIL:  drop cascades to view patch1_pixels
drop cascades to view patch1_tiles
NOTICE:  drop cascades to 2 other objects
DETAIL:  drop cascades to view patch2_pixels
drop cascades to view patch2_tiles
DROP TABLE


In [18]:
%%sql

CREATE TABLE patch1(
    rid SERIAL primary key,
    rast raster
);

-- insert empty raster into the base_raster table

INSERT INTO patch1(rast)
SELECT ST_AddBand( -- Returns a raster with the new band(s) of given type
  ST_MakeEmptyRaster( -- Returns an empty raster (having no bands)
      4, -- Raster width x (in pixels)
      4, -- Raster width y (in pixels)
      1.5, -- Upper left X coordinate
      -1.5, -- Upper left Y coordinate
      1, -- X Cell Resolution (in SRID units - in this case meters)
      -1, -- Y Cell Resolution (in SRID units - in this case meters)
      0, -- X skew
      0, -- Y skew
      3794 -- SRID
    ),
    
    ARRAY [
      ROW(
        1,  -- band index
        '8BUI'::text,  -- 8BUI - 8-bit unsigned integer (https://postgis.net/docs/RT_ST_BandPixelType.html)
        3,  -- initial pixel value
        0  -- NODATA value
      )
    ]::addbandarg[] -- a composite type used as input into the ST_AddBand function (https://postgis.net/docs/addbandarg.html)
);

SELECT AddRasterConstraints('patch1'::name, 'rast'::name);

CREATE TABLE patch2(
    rid SERIAL primary key,
    rast raster
);

-- insert empty raster into the base_raster table

INSERT INTO patch2(rast)
SELECT ST_AddBand( -- Returns a raster with the new band(s) of given type
  ST_MakeEmptyRaster( -- Returns an empty raster (having no bands)
      4, -- Raster width x (in pixels)
      4, -- Raster width y (in pixels)
      3.5, -- Upper left X coordinate
      -3.5, -- Upper left Y coordinate
      1, -- X Cell Resolution (in SRID units - in this case meters)
      -1, -- Y Cell Resolution (in SRID units - in this case meters)
      0, -- X skew
      0, -- Y skew
      3794 -- SRID
    ),
    
    ARRAY [
      ROW(
        1,  -- band index
        '8BUI'::text,  -- 8BUI - 8-bit unsigned integer (https://postgis.net/docs/RT_ST_BandPixelType.html)
        3,  -- initial pixel value
        0  -- NODATA value
      )
    ]::addbandarg[] -- a composite type used as input into the ST_AddBand function (https://postgis.net/docs/addbandarg.html)
);

SELECT AddRasterConstraints('patch2'::name, 'rast'::name);

CREATE OR REPLACE FUNCTION myarray(_offset integer)
  RETURNS double precision[][] AS
\$func\$
DECLARE
    a double precision[][][4][4] := array_fill(0, ARRAY[4,4]);
BEGIN
	FOR i IN 1..4 LOOP
		FOR j IN 1..4 LOOP
			a[i][j]= _offset + (i-1)*4 + j;
		END LOOP;
	END LOOP;
   RETURN a;
END
\$func\$  LANGUAGE plpgsql;


UPDATE patch1
SET rast = ST_SetValues(rast,1,1,1,myarray(0));

UPDATE patch2
SET rast = ST_SetValues(rast,1,1,1,myarray(100));

UPDATE patch2
SET rast = ST_SetValue(rast, 1, 1, 0);

UPDATE patch2
SET rast = ST_SetBandNoDataValue(rast, 0);

SELECT visualize_tiles_and_pixels('patch1');
SELECT visualize_tiles_and_pixels('patch2');

NOTICE:  Adding SRID constraint
NOTICE:  Adding scale-X constraint
NOTICE:  Adding scale-Y constraint
NOTICE:  Adding blocksize-X constraint
NOTICE:  Adding blocksize-Y constraint
NOTICE:  Adding alignment constraint
NOTICE:  Adding number of bands constraint
NOTICE:  Adding pixel type constraint
NOTICE:  Adding nodata value constraint
NOTICE:  Adding out-of-database constraint
NOTICE:  Adding maximum extent constraint
NOTICE:  Adding SRID constraint
NOTICE:  Adding scale-X constraint
NOTICE:  Adding scale-Y constraint
NOTICE:  Adding blocksize-X constraint
NOTICE:  Adding blocksize-Y constraint
NOTICE:  Adding alignment constraint
NOTICE:  Adding number of bands constraint
NOTICE:  Adding pixel type constraint
NOTICE:  Adding nodata value constraint
NOTICE:  Adding out-of-database constraint
NOTICE:  Adding maximum extent constraint
 visualize_tiles_and_pixels 
----------------------------
 
(1 row)

